<a href="https://colab.research.google.com/github/salvapineda/notebooks/blob/main/DC_OPF_chance_constrained.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is the code for the chance cosntrained DC optimal power flow.

# Requirements

In [ ]:
!pip install -q gurobipy
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pandas as pd

# Data

In [ ]:
# generation data
gen = pd.DataFrame({        
       'bus':       [0,    1, 2],
       'cost':      [20,   30, 0], 
       'cost_up':   [30,   35, 0], 
       'cost_do':   [10,   25, 0], 
       'pmin':      [0,   0, 0], 
       'pmax':      [100,  50, 0]})

lin = pd.DataFrame({
       'line': [0,   1,   2], 
       'from': [0,   0,   1],
       'to':   [1,   2,   2], 
       'b':    [1,  1,  1], 
       'cap':  [60, 60, 60]})

# Number of elements in the system
nbus = max(max(lin['from']),max(lin['to']))+1
ngen = nbus
nlin = len(lin)

# Power Transfer Distribution Factor
A_max = np.zeros((nlin,nbus))
for l in range(nlin): 
    A_max[l,lin.loc[l,'from']] = 1
    A_max[l,lin.loc[l,'to']]   = -1 
node_ref = 0
A_m  = np.delete(A_max,node_ref,axis=1)
X_m  = np.diag(1/lin['b'].values)
B_m  = np.linalg.multi_dot([A_m.T,np.linalg.inv(X_m),A_m])
ptdf = np.linalg.multi_dot([np.linalg.inv(X_m),A_m,np.linalg.inv(B_m)])
PTDF = np.round(np.insert(ptdf,node_ref,np.zeros(nlin),axis=1),5)

# State of the system
demand = np.array([0,0,90])
var = np.array([0,0,20])      
scen = np.zeros((len(demand),11))
scen[2,:] = np.array([-1,-0.8,-0.6,-0.4,-0.2,0,0.2,0.4,0.6,0.8,1])
scen20    = 20*scen
demand_scenarios = np.reshape(demand,(3,1)) + scen20

# Sample average approximation of the joint chance-constrained DC-OPF

\begin{align}
\underset{p_n,\beta_n\geq0}{\min} \quad & \mathbb{E} \left[ \sum_n \left( c_n p_n + c^u_n(-\beta_n \Omega)^+ - c^d_n (\beta_n\Omega)^+ \right) \right]  \\
\text{s.t.} \quad &  \sum_n p_n - \hat{d}_n = 0 \\
& \sum_n \beta_n = 1\\
& \underline{p}_n \leq p_n\leq \overline{p}_n, \quad \forall n\\
& -\overline{f}_l \leq \sum_n B_{ln}\left(p_n  -\hat{d}_n \right) \leq \overline{f}_l, \quad \forall l\\
& -M\,z_s + \underline{p}_n \leq p_{n} - \Omega \beta_{n} \leq  \overline{p}_n+M\,z_s, \quad \forall n,s \\
& -M\,z_s -\overline{f}  \leq \sum_n B_{ln} \left(p_n - \Omega \beta_n + \hat{d}_n - e_n\,\eta_{ns}\right) \leq \overline{f}+M\,z_s, \quad \forall l,s\\
& \sum_s z_s  \leq \lfloor \epsilon \, |S| \rfloor\\
& z_s \in \{0,1\}, \quad \forall s
\end{align}

In [ ]:
def dc_opf_jcc(demand,var,scen,epsilon):
  nscen = scen.shape[1]
  omega = scen.sum(axis=0)
  M = 100
  # model
  m = gp.Model() 
  # variables
  power = m.addMVar(ngen,name='gen')
  beta  = m.addMVar(ngen,name='participation')
  z  = m.addMVar(nscen,vtype=GRB.BINARY,name='bin')
  # objective function
  m.setObjective(gen['cost'].values @ power + 0.25*var.sum()*(gen['cost_up'].values - gen['cost_do'].values) @ beta, GRB.MINIMIZE)
  # constraints
  m.addConstr(power.sum() == demand.sum())
  m.addConstr(beta.sum() == 1)
  m.addConstr(power >= gen['pmin'].values)
  m.addConstr(power <= gen['pmax'].values)
  m.addConstr(PTDF @ power >= -lin['cap'].values + PTDF @ demand)
  m.addConstr(PTDF @ power <=  lin['cap'].values + PTDF @ demand)
  m.addConstrs((power[g] - omega[s]*beta[g] >= gen['pmin'].values[g] - M*z[s] for g in range(ngen) for s in range(nscen)))
  m.addConstrs((power[g] - omega[s]*beta[g] <= gen['pmax'].values[g] + M*z[s] for g in range(ngen) for s in range(nscen)))
  m.addConstrs((PTDF[l,:] @ power - omega[s]*(PTDF[l,:] @ beta) - PTDF[l,:] @ demand + PTDF[l,:] @ scen[:,s] >= -lin['cap'].values[l] - M*z[s] for l in range(nlin) for s in range(nscen)))
  m.addConstrs((PTDF[l,:] @ power - omega[s]*(PTDF[l,:] @ beta) - PTDF[l,:] @ demand + PTDF[l,:] @ scen[:,s] <=  lin['cap'].values[l] + M*z[s] for l in range(nlin) for s in range(nscen)))
  m.addConstr(z.sum() <= int(nscen*epsilon))
  # solve
  m.setParam('OutputFlag',0)
  m.optimize()
  
  # Calculate the violation per constraint and scenario
  vio = pd.DataFrame()
  for s in range(nscen):
     plus =  list( -power.X + omega[s] * beta.X + gen['pmin'].values)
     plus += list(  power.X - omega[s] * beta.X - gen['pmax'].values)
     plus += list(-PTDF @ (power.X - omega[s] * beta.X - demand + scen[:,s]) - lin['cap'].values)
     plus += list( PTDF @ (power.X - omega[s] * beta.X - demand + scen[:,s]) - lin['cap'].values)
     vio[s] = plus    
  
  # results
  print('########## JCC #############')
  print('cost =',round(m.ObjVal,1))
  print('dispatch =',np.round(power.X,2))
  print('participation factor =',np.round(beta.X,2))
  print('violated scenarios =',np.round(z.X,0))
  return power.X, np.round(z.X,0)

# Real time operation

\begin{align}
\underset{r^u_n\geq 0,r^d_n\geq 0}{\min} \quad & \sum_n c_n p^*_n + c^u_n r^u_n - c^d_n r^d_n \\
\text{s.t.} \quad & \sum_n p^*_n+r^u_n-r^d_n = \sum_n \tilde{d}_n - e \eta_n\\
& \underline{p} \leq p^*_n + r^u_n-r^d_n \leq \overline{p}_n, \quad \forall n\\
& -\overline{f}_l \leq \sum_n b_{ln}(p^*_n + r^u_n-r^d_n -\tilde{d}_n) \leq \overline{f}_l, \quad \forall l \\
\end{align}

In [ ]:
def real_time(demand,dispatch):
  # model
  m = gp.Model()
  # variables
  res_up = m.addMVar(ngen)
  res_do = m.addMVar(ngen)
  # objective function
  m.setObjective(gen['cost'].values @ dispatch + gen['cost_up'].values @ res_up - gen['cost_do'].values @ res_do, GRB.MINIMIZE)
  # constraints
  m.addConstr(dispatch.sum() + res_up.sum() - res_do.sum() == demand.sum())
  m.addConstr(dispatch + res_up - res_do >= gen['pmin'].values)
  m.addConstr(dispatch + res_up - res_do <= gen['pmax'].values)
  m.addConstr(PTDF @ dispatch + PTDF @ res_up - PTDF @ res_do - PTDF @ demand >= -lin['cap'].values)
  m.addConstr(PTDF @ dispatch + PTDF @ res_up - PTDF @ res_do - PTDF @ demand <=  lin['cap'].values)
  # solve
  m.setParam('OutputFlag',0)
  m.optimize()
  return m.ObjVal

In [ ]:
def evaluate_scenarios(dispatch,demand_scenarios):
  v_cost = []
  for s in range(demand_scenarios.shape[1]):
    obj = real_time(demand_scenarios[:,s],dispatch)
    v_cost.append(obj)    
  print('minimum cost =',round(np.min(v_cost),1))
  print('average cost =',round(np.mean(v_cost),1))
  print('maximum cost =',round(np.max(v_cost),1))

# Evaluate the SAA JCC-DC-OPF when $\epsilon = 0.0$

In [ ]:
gen_jcc,z_var = dc_opf_jcc(demand,var,scen20,0.0)
evaluate_scenarios(np.round(gen_jcc,2), demand_scenarios)

########## JCC #############
cost = 2050.0
dispatch = [70. 20.  0.]
participation factor = [0. 1. 0.]
violated scenarios = [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
minimum cost = 1500.0
average cost = 2043.6
maximum cost = 2700.0


# Evaluate the SAA JCC-DC-OPF when $\epsilon = 0.2$

In [ ]:
gen_jcc,z_var = dc_opf_jcc(demand,var,scen20,0.2)
evaluate_scenarios(np.round(gen_jcc,2), demand_scenarios[:,1:10])

########## JCC #############
cost = 2010.0
dispatch = [74. 16.  0.]
participation factor = [0. 1. 0.]
violated scenarios = [ 1.  0.  0. -0. -0.  0. -0. -0. -0.  0.  1.]
minimum cost = 1560.0
average cost = 1995.6
maximum cost = 2520.0
